# Lab 5. TensorFlow and Convolutional Neural Network

In [ ]:
from tqdm.keras import TqdmCallback
from tqdm import tqdm_notebook
from livelossplot import PlotLossesKeras
from keras.optimizers import SGD
from keras.utils import plot_model
import keras
from IPython.display import Image

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import matplotlib.pyplot as plt
import visualkeras as vk
from PIL import Image
CPU_ONLY = False

In [ ]:
if CPU_ONLY: os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
print(f"TensorFlow version: {tf.__version__}")
print("CUDA version:")
print(os.popen('nvcc --version').read())
with_cuda = tf.test.is_built_with_cuda()
print(f"Can build with CUDA: {with_cuda}")
gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus: tf.config.experimental.set_memory_growth(gpu, True)
print("Num GPUs Available: ", len(gpus))
for x in gpus: print(x)

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

In [ ]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [ ]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))
num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))
total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

In [ ]:
print('total training cat images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)
print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

In [ ]:
batch_size = 128
epochs = 20
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
directory=train_dir, shuffle=True, target_size=(IMG_HEIGHT, IMG_WIDTH),
class_mode='binary')

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
directory=validation_dir,target_size=(IMG_HEIGHT, IMG_WIDTH),class_mode='binary')

In [ ]:
sample_training_images, _ = next(train_data_gen)
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
plotImages(sample_training_images[:5])

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [ ]:
model.compile(optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'])
model.summary()
vk.layered_view(model, spacing=30)

In [ ]:
pbar = TqdmCallback(verbose=1,tqdm_class=tqdm_notebook, leave = True, display = False)
pbar.epoch_bar.ncols=0
plot = PlotLossesKeras()

In [ ]:
%%time
pbar.display()
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    verbose=0,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    callbacks=[pbar,plot]
)

In [ ]:
def plotResult(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss=history.history['loss']
    val_loss=history.history['val_loss']
    tla = []
    vla = []
    for i in range(len(acc)): 
        tla.append(acc[i]*loss[i])
        vla.append(val_acc[i]*val_loss[i])
    epochs_range = range(epochs)
    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()
plotResult(history)

## Q1. 

How would you describe the trend of (1) training accuracy, (2) validation accuracy, (3) training loss, and (4) validation loss?

- Training accuracy: Upward trending 
- Validation accuracy: Rise quickly then oscillates around 0.73
- Training loss: Downward trending 
- Validation loss: Decreased to around 0.6, follow with a rebound and is then gradually increasing

## Q2. 

Do you observe any issue in the plots, for example, overfitting?

The model is overfitting to the training data, whereas the accuracy for testing data is stagnant. Other than that, the gradually increasing validation loss indicates that the model is starting to "memorize" the training data. 

![](cute_puppy.jpg)

In [ ]:
image_gen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True,
    zoom_range=0.5
)
train_data_gen = image_gen_train.flow_from_directory(
    batch_size=batch_size,
    directory=train_dir, 
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary')

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

In [ ]:
image_gen_val = ImageDataGenerator(rescale=1./255)
val_data_gen = image_gen_val.flow_from_directory(
    batch_size=batch_size,
    directory=validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary')

# Assignment 

In [ ]:
modelImproved = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

In [ ]:
modelImproved.compile(optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'])
modelImproved.summary()
vk.layered_view(modelImproved, spacing=30)

In [ ]:
pbar = TqdmCallback(verbose=1,tqdm_class=tqdm_notebook, leave = True, display = False)
pbar.epoch_bar.ncols=0
plot = PlotLossesKeras()
epochs = 175

In [ ]:
%%time
pbar.display()
history = modelImproved.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    verbose=0,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    callbacks=[pbar,plot]
)

In [ ]:
plotResult(history)

## Q3. 

How would you describe the trend of (1) training accuracy, (2) validation accuracy, (3) training loss, and (4) validation loss?

- Training accuracy: Upward trending 
- Validation accuracy: Rise along with training accuracy line, oscillates around 0.78
- Training loss: Downward trending 
- Validation loss: Decrease along with training loss line, oscillates around 0.5

## Q4. 

Do you think the issue before regularization is solved after regularization?

To some degrees, yes, the model is less overfitted after regularization. However, the validation accuracy and validation loss still shows that there could be more improvements.

![](cute_kitpy.jpg)

# ARTIFICIAL STUPIDITY

In [ ]:
epochs = 200
accuracy_threshold = 0.98
batch_size = 64

In [ ]:
train_image_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.5,
    zoom_range=0.5,
    horizontal_flip=True,
    vertical_flip=True)

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=train_dir, 
    shuffle=True, 
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary')
val_data_gen = validation_image_generator.flow_from_directory(
    batch_size=batch_size,
    directory=validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='binary')

In [ ]:
magic = Sequential()
magic.add(Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)))

magic.add(Conv2D(64, (3, 3), padding='same', activation='relu', name='block1_conv1'))
magic.add(Conv2D(64, (3, 3), padding='same', activation='relu', name='block1_conv2'))
magic.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))
magic.add(vk.SpacingDummyLayer(spacing=100))

magic.add(Conv2D(128, (3, 3), padding='same', activation='relu', name='block2_conv1'))
magic.add(Conv2D(128, (3, 3), padding='same', activation='relu', name='block2_conv2'))
magic.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))
magic.add(vk.SpacingDummyLayer(spacing=100))

magic.add(Conv2D(256, (3, 3), padding='same', activation='relu', name='block3_conv1'))
magic.add(Conv2D(256, (3, 3), padding='same', activation='relu', name='block3_conv2'))
magic.add(Conv2D(256, (3, 3), padding='same', activation='relu', name='block3_conv3'))
magic.add(Conv2D(256, (3, 3), padding='same', activation='relu', name='block3_conv4'))
magic.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))
magic.add(vk.SpacingDummyLayer(spacing=100))

magic.add(Conv2D(512, (3, 3), padding='same', activation='relu', name='block4_conv1'))
magic.add(Conv2D(512, (3, 3), padding='same', activation='relu', name='block4_conv2'))
magic.add(Conv2D(512, (3, 3), padding='same', activation='relu', name='block4_conv3'))
magic.add(Conv2D(512, (3, 3), padding='same', activation='relu', name='block4_conv4'))
magic.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))
magic.add(vk.SpacingDummyLayer(spacing=100))

magic.add(Conv2D(512, (3, 3), padding='same', activation='relu', name='block5_conv1'))
magic.add(Conv2D(512, (3, 3), padding='same', activation='relu', name='block5_conv2'))
magic.add(Conv2D(512, (3, 3), padding='same', activation='relu', name='block5_conv3'))
magic.add(Conv2D(512, (3, 3), padding='same', activation='relu', name='block5_conv4'))
magic.add(MaxPooling2D((2, 2), padding='same', strides=(2, 2), name='block5_pool'))
magic.add(vk.SpacingDummyLayer(spacing=100))

magic.add(Flatten(name='flat'))
magic.add(Dense(1024, activation='relu', name='dense_1024'))
magic.add(Dropout(0.5))
magic.add(Dense(512, activation='relu', name='dense_512'))
magic.add(Dropout(0.25))
magic.add(Dense(128, activation='relu', name='dense_128'))
magic.add(Dropout(0.1))
magic.add(Dense(1))

magic.load_weights('weights.h5', by_name=True,skip_mismatch=True)

In [ ]:
magic.compile(
    optimizer=SGD(lr=1e-4, momentum=0.9, nesterov=True),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy'])
print(magic.summary())
vk.layered_view(magic, spacing=30)

In [ ]:
class AccuracyStopping(keras.callbacks.Callback):
    def __init__(self, acc_threshold, verbose=1):
        super(AccuracyStopping, self).__init__()
        self._acc_threshold = acc_threshold
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        train_acc = logs.get('accuracy') != None and logs.get('accuracy')
        if self.verbose > 0: print(f'Training Accuracy Threshold: {train_acc} / {self._acc_threshold}')
        self.model.stop_training = train_acc >= self._acc_threshold
acc_callback = AccuracyStopping(accuracy_threshold, verbose=0)
pbar = TqdmCallback(verbose=1,tqdm_class=tqdm_notebook, leave = True, display = False)
pbar.epoch_bar.ncols=0
plot = PlotLossesKeras()

In [ ]:
%%time
pbar.display()
history = magic.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    verbose=0,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    callbacks=[pbar,plot,acc_callback]
)

![](cute_kitten.jpg)

In [ ]:
magic.save_weights('magic_weights.h5')